# 第一步：导入需要的库

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 第二步：读取文件

In [ ]:
# original_file = pd.read_csv(" ")
# original_file.head()

# 第三步：评估数据  

在这一部分中，我们将对在上一部分建立的original_file DataFrame所包含的数据进行评估和清理。  

主要从两个方面进行：结构和内容，即整齐度和干净度。  

数据的结构性问题指不符合“每个变量为一列，每个观察值为一行，每种类型的观察单位为一个表格”这三个标准；数据的内容性问题包括存在丢失数据、重复数据、无效数据等。  

为了区分开经过清理的数据和原始的数据，我们创建新的变量cleaned_file，让它为original_file复制出的副本。我们之后的清理步骤都将被运用在cleaned_file上。  

In [ ]:
# cleaned_file = original_file.copy()

数据整齐度（即每个变量为一列，每个观察值为一行，每种类型的观察单位为一个表格）

In [4]:
# cleaned_file.copy()

数据干净度（用info进行初步观察）

In [ ]:
# cleaned_file.info()

从输出结果来看，找到存在缺失值的变量名，将在后续进行评估和清理。

从数据类型来看，应对某些类型的数据进行类型转换。

同时分类数据都应转换成Category

In [ ]:
# cleaned_file['列名'] = cleaned_file['前面的列名'].astype('数据类型')

再次通过info查看数据

In [5]:
# cleaned_file.info()

# 第四步：处理缺失数据

看NaN的值有多少，即为有多少缺失值

In [ ]:
# cleaned_file[cleaned_file['缺数据的列名'].isna()]

处理缺失值方法：

In [ ]:
# average_data = cleaned_file['缺数据的列名'].mean()   算平均数
# cleaned_file['缺数据的列名'] = cleaned_file['缺数据的列名'].fillna(average_data) 将缺失值用平均数替换
#cleaned_file['缺数据的列名'].isna().sum()        如果显示0，则处理完成

# 第五步：处理重复数据

In [6]:
# cleaned_file["重复变量名"].duplicated().sum()   输出结果应为0，否则表示有重复值

# 第六步：处理不一致数据

不一致数据可能存在于所有分类变量中，我们要查看是否存在不同值实际指代同一目标的情况。

In [8]:
# cleaned_file["变量名"].value_counts

# 第七步：处理无效或错误数据

可以先通过DataFrame的describe（）方法，对数值统计信息进行快速了解。

In [10]:
# cleaned_file.describe()

# 第八步：整理数据

包括将一些变量名进行合并，拆分等操作

In [12]:
# cleaned_file['合并后的变量名'] = cleaned_file['变量名1'] + cleaned_file['变量名2']
# cleaned_file.head()

探索数据

在着手逻辑回归分析之前，我们可以先借助数据可视化，探索数值变量的分布，以及与乘客是否幸存存在相关性的变量，为后续的进一步分析提供方向。

In [13]:
# 设置图表色盘为"pastel"，也可以是其他色盘
# sns.set_palette("pastel")

In [ ]:
# 设置图表尺寸，不同尺寸都可以，取汝之喜好即可
# plt.rcParams["figure.figsize"] = [7.00, 3.50]
# plt.rcParams["figure.autolayout"] = True

绘制饼图

In [ ]:
# data1_count = cleaned_file['变量名'].value_counts()
# data2_label = data1.index
# plt.pie(data1_count, labels=data2_label, autopct='%.1f%%')
# plt.show()

绘制直方图和箱式图

In [ ]:
# figure, axes = plt.subplots(1, 2)
# sns.histplot(cleaned_file, x='变量名', ax=axes[0])
# sns.boxplot(cleaned_file, y='变量名', ax=axes[1])
# plt.show()

# 第九步：分析数据

In [15]:
# import statsmodels.api as sm

然后可以创建一个新的DataFrame lr_file，让它作为我们进逻辑性回归分析所用的数据。

和cleaned_file区分开的原因是，我们在进行回归分析前，还可能需要对数据进行一些准备。

比如引入虚拟变量，这些都可以在lr_file上执行。

In [16]:
# lr_file = cleaned_file.copy()   备份
# lr_file.head()                  初步观察

移除大概率不会影响研究的变量。

In [19]:
# lr_file = lr_file.drop(['无关变量1', '无关变量2', '无关变量3', ......, '无关变量n'], axis=1)
# lr_file.head()    观察删除后的数据

数据里还存在分类变量，无法直接建立逻辑回归模型。我们需要引入虚拟变量，也就是用0和1分别表示是否属于该类别。

用get_dummies方法

In [21]:
# lr_file = pd.get_dummies(lr_file, drop_first=True, columns=['分类变量1', ...... , '分类变量n'], dtype=int)
# lr_file.head()

接下来，我们要把因变量（y）和自变量（x）划分出来。

In [ ]:
# y = lr_file['因变量的列名']

我们可以把除 '因变量对应列名' 之外的先纳入自变量，但需要查看它们之间的相关性。

如果其中有些变量之间相关性很高，会导致共线性。

In [ ]:
# X = lr_file.drop(['Survived'], axis=1)  这里用drop方法将因变量的列名删除后，其余列名赋值成自变量
# X.corr()      用corr方法观察自变量之间的相关性强弱

一般我们认为，当相关系数的绝对值大于0.8的时候，可能导致严重共线性，所以我们检查的时候，找绝对值大于0.8的值即可。

In [ ]:
# X.corr().abs() > 0.8

不同变量之间的如果相关性过高，会导致数值优化算法无法收敛，无法获得逻辑回归模型参数的计算结果。

此外，需仔细看相关系数数值，如果某些自变量的相关系数接近0.8，也应对其进行移除，避免算法无法收敛。

In [22]:
# X = X.drop(['自变量1', ...... , '自变量n'], axis=1)  删除相关性过高的自变量

接下来，给模型的线性方程添加截距。

用sm库里面的add_constant函数

In [23]:
# X = sm.add_constant(X)

下一步就可以调用Logit函数，利用最大似然优化来得到逻辑回归模型的参数值，并输出总结信息。

In [ ]:
# model = sm.Logit(y, X).fit()
# model.summary()

当我们把显著区间设定为0.05时，以上结果的P值，如果发现某个自变量对研究无影响（即P值大于0.05）时，应该把这个变量移除后，再次建立逻辑回归模型。

In [ ]:
# X = X.drop(['无影响的变量名'], axis=1)
# model = sm.Logit(y, X).fit()
# model.summary()

要理解各个各个自变量系数的实际含义，我们需要计算自然常数的次方。

直接用numpy库里面的exp

In [ ]:
# np.exp(系数)

# 第十步：根据模型推数据
根据样本数据建立模型后，我们就可以用此模型去推其他数据啦。

In [26]:
# data_test = pd.read_csv("想要预测数据的文件")
# data_test.head()

由于逻辑回归模型不允许数据中有缺失值，因此我们需要检查data_test是否存在数据缺失。

In [28]:
# data_test.info()

缺失值若不属于回归模型的自变量，可以忽略

若属于回归模型的自变量，可以用平均值进行填充

In [30]:
# data_test['缺数据的变量名'] = data_test['缺数据的变量名'].fillna(data_test['缺数据的变量名'].mean())
# data_test['缺数据的变量名'].isna().sum()

In [ ]:
下一步是给模型中的分类变量引入虚拟变量，但在引入前我们需要先把分类变量的类型转换为Category，并且通过categories参数，让程序知道所有可能的分类值。

这样做的原因是，预测数据包含的分类可能不全。我们需要确保引入虚拟变量的时候，不会漏掉某个或某些分类。

In [31]:
# data_test['分类变量1'] = pd.Categorical(data_test['分类变量1'], categories=[所有类型])
# data_test['分类变量2'] = pd.Categorical(data_test['分类变量2'], categories=[所有类型])
# data_test['分类变量n'] = pd.Categorical(data_test['分类变量3'], categories=[所有类型])

下一步，给模型用到的分类变量引入虚拟变量。

还是get_dummies方法

In [ ]:
# data_test = pd.get_dummies(data_test, drop_first=True, columns=['分类变量1', ......, '分类变量n'], dtype=int)
# data_test.head()

查看一下模型需要的输入变量。

In [ ]:
# model.params

接下来构建我们要输入给模型进行预测的变量，需要和模型训练时的输入一致。

In [ ]:
# X_test = data_test[[......]]
# X_test = sm.add_constant(X_test)

现在就可以调用逻辑回归模型的predict方法，获得预测值。

In [ ]:
# predicted_value = model.predict(X_test)
# predicted_value